In [1]:
# Cell 1: 导入库
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
import datetime
import re
import os

In [ ]:
# Cell 2: 爬虫类
class BossSpiderAPI:
    def __init__(self):
        self.base_url = "https://www.zhipin.com/wapi/zpgeek/search/joblist.json"
        
        # 从文件读取 Cookie
        cookie = self._load_cookie_from_file()
        
        # 固定 User-Agent，减少变化带来的风险
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
            "Referer": "https://www.zhipin.com/web/geek/job",
            "Accept": "application/json, text/plain, */*",
            "Cookie": cookie
        }
        
        self.finance_pattern = re.compile(
            r"(金融|银行|券商|证券|基金|保险|期货|信托|投行|资管|理财|财富|小贷|消金|保理|租赁|征信|清算|支付|互金|金融科技|FinTech)",
            re.IGNORECASE
        )
        
        self.progress_file = "job_data_IN_PROGRESS.xlsx"
        self.state_file = "crawl_state.txt"
        self.detail_state_file = "detail_state.txt"
        self.data_list = []
        self.seen_ids = set()

    def _load_cookie_from_file(self, filepath="cookie.txt"):
        """从文件读取 Cookie"""
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                cookie = f.read().strip()
                # 过滤掉注释行
                lines = [line.strip() for line in cookie.split('\n') 
                         if line.strip() and not line.strip().startswith('//')]
                cookie = ''.join(lines)
                if cookie:
                    print(f"✅ 成功从 {filepath} 加载 Cookie")
                    return cookie
                else:
                    print(f"⚠️ {filepath} 文件为空")
                    return ""
        except FileNotFoundError:
            print(f"⚠️ 未找到 {filepath}，请创建该文件并填入 Cookie")
            return ""
        except Exception as e:
            print(f"❌ 读取 Cookie 文件失败: {e}")
            return ""

    def _simple_sleep(self, min_sec=2.0, max_sec=4.0):
        """简单的随机等待"""
        time.sleep(random.uniform(min_sec, max_sec))

    def is_finance_related(self, job):
        """判断职位是否与金融相关"""
        check_fields = [
            job.get("jobName", ""),
            job.get("brandName", ""),
            job.get("brandIndustry", ""),
        ]
        combined_text = " ".join(str(f) for f in check_fields if f)
        return bool(self.finance_pattern.search(combined_text))

    def load_crawl_state(self, keywords):
        """加载爬取状态"""
        if not os.path.exists(self.state_file):
            return 0, 1
        
        try:
            with open(self.state_file, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                if len(lines) < 2: return 0, 1
                keyword = lines[0].strip()
                page = int(lines[1].strip())
                
                if keyword in keywords:
                    idx = keywords.index(keyword)
                    print(f"📍 恢复列表爬取进度：关键词 [{keyword}] 第 {page} 页")
                    return idx, page
                else:
                    return 0, 1
        except Exception as e:
            print(f"⚠️ 读取状态文件失败: {e}，从头开始")
            return 0, 1

    def save_crawl_state(self, keyword, page):
        """保存爬取状态"""
        try:
            with open(self.state_file, 'w', encoding='utf-8') as f:
                f.write(f"{keyword}\n{page}")
        except Exception:
            pass

    def load_detail_state(self):
        """加载详情抓取进度"""
        if not os.path.exists(self.detail_state_file):
            return 0
        try:
            with open(self.detail_state_file, 'r', encoding='utf-8') as f:
                index = int(f.read().strip())
                print(f"📍 恢复详情抓取进度：从第 {index + 1} 条开始")
                return index
        except Exception:
            return 0

    def save_detail_state(self, index):
        """保存详情抓取进度"""
        try:
            with open(self.detail_state_file, 'w', encoding='utf-8') as f:
                f.write(str(index))
        except Exception:
            pass

    def fetch_data(self, query="金融", start_page=1, max_pages=5):
        """抓取职位列表 - 简化版"""
        consecutive_fail = 0
        
        for page in range(start_page, max_pages + 1):
            print(f"\n🔍 [{query}] 第 {page} 页...")
            
            params = {
                "scene": "1", "query": query, "city": "100010000",
                "page": page, "pageSize": 30
            }
            
            try:
                resp = requests.post(self.base_url, headers=self.headers, data=params, timeout=15)
                result = resp.json()
                code = result.get("code")
                
                if code == 37:
                    print(f"⚠️ 风控触发！保存进度后退出")
                    self.save_crawl_state(query, page)
                    self.save_progress()
                    return False
                    
                if code not in (0, "0", None):
                    print(f"⚠️ 异常 code={code}")
                    consecutive_fail += 1
                    if consecutive_fail >= 5:
                        break
                    self._simple_sleep(5, 10)
                    continue
                    
                consecutive_fail = 0  # 重置失败计数
                
            except Exception as e:
                print(f"❌ 请求失败: {e}")
                consecutive_fail += 1
                if consecutive_fail >= 5:
                    break
                self._simple_sleep(3, 6)
                continue

            job_list = result.get("zpData", {}).get("jobList", [])
            if not job_list:
                print("⚠️ 无数据，跳过")
                break
            
            finance_count = 0
            for job in job_list:
                job_id = job.get("encryptJobId")
                if not job_id or job_id in self.seen_ids:
                    continue
                if not self.is_finance_related(job):
                    continue
                
                finance_count += 1
                self.data_list.append({
                    "职位": job.get("jobName"),
                    "公司": job.get("brandName"),
                    "薪资": job.get("salaryDesc"),
                    "地区": job.get("cityName"),
                    "经验": job.get("jobExperience"),
                    "学历": job.get("jobDegree"),
                    "公司规模": job.get("brandScaleName"),
                    "行业": job.get("brandIndustry"),
                    "福利标签": ",".join(job.get("welfareList", []) or []),
                    "技能标签": ",".join(job.get("skills", []) or []),
                    "职位描述": "",
                    "job_id": job_id
                })
                self.seen_ids.add(job_id)
            
            print(f"✅ 本页金融相关: {finance_count} | 累计: {len(self.data_list)}")
            
            self.save_crawl_state(query, page + 1)
            self.save_progress()
            
            # 简化等待：固定 2-4 秒
            self._simple_sleep(2.0, 4.0)
        
        return True

    def get_job_detail(self, job_id):
        """获取职位详情描述"""
        url = f"https://www.zhipin.com/job_detail/{job_id}.html"
        
        try:
            resp = requests.get(url, headers=self.headers, timeout=15)
            
            if resp.status_code == 404:
                return "职位已关闭或不存在"
            if resp.status_code == 403:
                print(f"  ⚠️ 403 禁止访问")
                return None
            if resp.status_code != 200:
                print(f"  ⚠️ 状态码: {resp.status_code}")
                return None
            
            if 'security-check' in resp.url or 'verify' in resp.url:
                print(f"  ⚠️ 触发安全验证跳转")
                return None
            
            soup = BeautifulSoup(resp.text, 'html.parser')
            
            selectors = [
                '.job-sec-text', '.job-detail-section .text', '.job-sec .text',
                '.job-description', '.job-detail .text', '[class*="job-sec"] .text'
            ]
            
            for selector in selectors:
                desc_tag = soup.select_one(selector)
                if desc_tag and desc_tag.text.strip():
                    return desc_tag.text.strip()
            
            if "职位描述" in resp.text:
                print(f"  ⚠️ 页面含关键词但未匹配到选择器")
            return ""
            
        except requests.exceptions.RequestException as e:
            print(f"  ⚠️ 详情页请求异常: {e}")
            return None

    def load_progress(self):
        """从进度文件中加载数据"""
        print(f"\n🔄 正在从 {self.progress_file} 加载进度...")
        try:
            df = pd.read_excel(self.progress_file)
            df['职位描述'] = df['职位描述'].fillna('')
            df['job_id'] = df['job_id'].fillna('')
            
            self.data_list = df.to_dict('records')
            self.seen_ids = {item['job_id'] for item in self.data_list if item.get('job_id')}
            
            print(f"✅ 加载成功，已恢复 {len(self.data_list)} 条记录。")
            
        except FileNotFoundError:
            print("ℹ️ 未找到进度文件，将创建新文件。")
        except Exception as e:
            print(f"❌ 加载进度文件失败: {e}")

    def save_progress(self):
        """将当前数据保存到进度文件"""
        if not self.data_list:
            return
        try:
            df = pd.DataFrame(self.data_list)
            df.to_excel(self.progress_file, index=False, na_rep='')
        except Exception as e:
            print(f"  ❌ 保存进度失败: {e}")

    def fetch_all_details_resumable(self, start_index=0):
        """可恢复地抓取所有详情"""
        print(f"\n{'='*60}")
        print(f"🚀 (阶段2) 开始抓取职位详细描述...")
        if start_index > 0:
            print(f"📍 跳过前 {start_index} 条，直接从第 {start_index + 1} 条开始")
        print(f"{'='*60}")
        
        fetched_count = 0
        consecutive_fail = 0
        total_items = len(self.data_list)
        
        for i in range(start_index, total_items):
            item = self.data_list[i]
            
            desc_str = str(item.get("职位描述", "")).strip()
            if desc_str and desc_str.lower() != 'nan' and len(desc_str) > 10:
                continue
            
            print(f"  ({i+1}/{total_items}) [抓取] {item['职位']} - {item['公司']}")
            
            job_id = item.get("job_id")
            if not job_id:
                continue
            
            new_desc = self.get_job_detail(job_id)
            
            if new_desc is None:
                consecutive_fail += 1
                print(f"  ❌ 抓取失败 (连续: {consecutive_fail})")
                self.save_detail_state(i)
                
                if consecutive_fail >= 3:
                    print(f"\n  🛑 连续失败触发风控保护，停止运行。")
                    self.save_progress()
                    return i
                
                self._simple_sleep(5.0, 10.0)
                
            elif new_desc == "":
                consecutive_fail = 0
                item["职位描述"] = "[未获取到描述]"
                print(f"  ⚠️ 未找到描述内容")
                self._simple_sleep(3.0, 5.0)
                
            else:
                consecutive_fail = 0
                item["职位描述"] = new_desc
                fetched_count += 1
                print(f"  ✅ 获取成功")
                
                if fetched_count % 5 == 0:
                    self.save_progress()
                    self.save_detail_state(i + 1)
                
                self._simple_sleep(4.0, 8.0)
        
        self.save_progress()
        self.save_detail_state(total_items)
        
        print("\n✅ (阶段2) 详情抓取完毕。")
        if os.path.exists(self.detail_state_file):
            os.remove(self.detail_state_file)
        
        return -1

    def save_final_excel(self):
        """保存为最终的 Excel 文件"""
        if not self.data_list:
            return
        
        final_data = [item.copy() for item in self.data_list]
        for item in final_data:
            item.pop('job_id', None)
                
        df = pd.DataFrame(final_data)
        filename = f"金融行业岗位_完整版_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"
        df.to_excel(filename, index=False)
        print(f"\n🎉 全部完成！已保存：{filename} (共 {len(df)} 条)")

    def run(self, keywords, max_pages_per_kw=10, skip_list=False):
        """主运行逻辑"""
        if os.path.exists(self.progress_file):
            self.load_progress()
        
        if not skip_list:
            start_kw_idx, start_page = self.load_crawl_state(keywords)
            
            print(f"\n🚀 (阶段1) 职位列表抓取")
            for kw_idx in range(start_kw_idx, len(keywords)):
                keyword = keywords[kw_idx]
                page = start_page if kw_idx == start_kw_idx else 1
                
                print(f"\n>>> 正在抓取: {keyword} ({kw_idx+1}/{len(keywords)})")
                success = self.fetch_data(query=keyword, start_page=page, max_pages=max_pages_per_kw)
                
                if not success:
                    print("\n⚠️ 程序暂停：请更新 Cookie 后重新运行")
                    return
                
                self._simple_sleep(3.0, 6.0)
            
            if os.path.exists(self.state_file):
                os.remove(self.state_file)
        
        start_detail_index = self.load_detail_state()
        if self.data_list:
            result = self.fetch_all_details_resumable(start_index=start_detail_index)
            if result >= 0:
                print("\n⚠️ 详情抓取暂停：请更新 Cookie 后重新运行")
                return
        
        self.save_final_excel()

In [11]:
# Cell 3: 运行爬虫
if __name__ == "__main__":
    spider = BossSpiderAPI()
    
    # 优化：去除了重复的关键词
    raw_keywords = ["银行", "保险", "数据分析", "基金经理", "产品", "运营", "证券研究所", "投行", "金融"]

    # 正常运行
    spider.run(keywords=raw_keywords, max_pages_per_kw=30)


🔄 正在从 job_data_IN_PROGRESS.xlsx 加载进度...
✅ 加载成功，已恢复 1979 条记录。
📍 恢复列表爬取进度：关键词 [银行] 第 10 页

🚀 (阶段1) 职位列表抓取

>>> 正在抓取: 银行 (1/9)

🔍 [银行] 第 10 页...
⚠️ 风控触发！保存进度后退出

⚠️ 程序暂停：请更新 Cookie 后重新运行
